In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import urllib
import re
import datetime
import pandas as pd
import numpy as np

In [2]:
def get_data(tags,x):
    #function to extract player details from their individual homepage.
    #returns a list of player details to be added to overall dataframe.
    
    player_name = str(tags[1].h2.a.string)[1:]
    player_id = int(tags[3+x].string)
    player_gen = str(tags[5+x].string)
    player_league = tags[7+x].string
    join_date = pd.to_datetime(tags[9+x].string)
    county = str(tags[11+x].string).split(',')[0]
    state = str(tags[11+x].string).split(',')[1]
    lg = tags[13+x].string
    if lg == 'Has not played':
        last_game = None
    else:
        last_game = pd.to_datetime(lg)
    allb = tags[17+x].find_all('a')
    if allb == 'INACTIVE':
        active_divisions = None
    else:
        active_divisions = [int(tg.string) for tg in allb]
    total_matches = float(tags[20+x].string)
    wins = float(str(tags[22+x].string).split('wins')[0])
    losses = total_matches-wins
    avgppm = float(tags[26+x].string)
    if total_matches == 0:
        win_percent = 0
    else:
        win_percent = wins/total_matches * 100
    return [player_name, player_id, player_gen, player_league, join_date, county, state, last_game, active_divisions,
            total_matches, wins, losses, avgppm, win_percent]

In [3]:
n = 10                              #number of pages to read
pid1 = 10062000                     #player ID at which to begin scrape
players = pd.DataFrame()

for i in range(0,n):
    #read each webpage, determine if data is missing, or page layout is unconventional
    #VIP players are ignored due to variable page layout
    #if data is readable, use get_data function 
    
    pid = pid1 + i
    add = 'https://www.napaleagues.com/stats.php?playerSelected=Y&playerID=' + str(pid)
    r = urllib.urlopen(add).read()
    soup = BeautifulSoup(r, 'lxml')
    tags = soup.find_all('td')      #all relevant data is contained within 'td' tag
    if 'wins' in str(tags[4].string):
        players[i]=[None]*14
    elif tags[3].find_all('strong') != []:
        #players[i]=get_data(tags,1)
        players[i]=[None]*14
    elif tags[3].find_all('font') != []:
        players[i]=[None]*14
    else:
        players[i]=get_data(tags,0)

#export dataframe to csv file
players = players.T
players.columns = ['Name','ID','Gender','League','Join Date','County','State','Last Game','Active Divisions','Total Matches',
                  'Won','Lost','AvgPPM','Win %']
add = r'C:\\Users\\Owner\\Napa\\players' + str(pid1) + '_' + str(n) +'.csv'
export_players=players.to_csv(add)
players.tail()

,Name,ID,Gender,League,Join Date,County,State,Last Game,Active Divisions,Total Matches,Won,Lost,AvgPPM,Win %
5,Jason Fayman,10062005,Male,NAPA of Montgomery,2019-06-09 00:00:00,Montgomery,Alabama,2019-07-01 00:00:00,[],1,0,1,7,0
6,Ricky Ledbetter,10062006,Male,NAPA of Montgomery,2019-06-09 00:00:00,Montgomery,Alabama,2019-08-26 00:00:00,[],9,7,2,12.22,77.7778
7,Greg Turner,10062007,Male,NAPA of Corpus Christi,2019-06-09 00:00:00,Aransas,Texas,2019-12-04 00:00:00,"[7689, 8002, 8030]",21,6,15,6.24,28.5714
8,Jerome Lawrence,10062008,Male,NAPA of Franklin,2019-06-09 00:00:00,Franklin,Missouri,2019-08-26 00:00:00,[],8,2,6,6.75,25
9,Gary Tang,10062009,Male,NAPA of San Diego,2019-06-09 00:00:00,San Diego,California,2019-09-09 00:00:00,[],8,1,7,5.88,12.5


In [5]:
import glob
import csv

#combine all csv files into one

delimiter = ','
csvs = glob.glob('C:\\Users\\Owner\\Napa\\*.csv')
combined_csv = pd.concat([pd.read_csv(f) for f in csvs])
combined_csv.to_csv('C:\\Users\\Owner\\Napa\\combined.csv')